In [6]:
import os

import librosa, librosa.display
import IPython.display as ipd
from random import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import signal
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

PROCESSED_DATA_PICKLE_PATH = '../data/df_all.pkl'
PROCESSED_DATA_PICKLE_PATH_N = '../data/df_normal.pkl'

EMOTIONS = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

MFCC_LENGTH = 125

In [8]:
df = pd.read_pickle(PROCESSED_DATA_PICKLE_PATH)
df.head()

,path,source,actor,gender,gender_label,statement,repetition,intensity,emotion,emotion_label,mfcc
0,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,1,0,0,neutral,"[[-857.3094533443688, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,2,0,0,neutral,"[[-864.8902862773604, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,1,1,0,0,neutral,"[[-849.4454325616318, 9.397479238778757, 9.257..."
3,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,1,2,0,0,neutral,"[[-832.7343966188961, 11.492822043371124, 0.14..."
4,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,1,0,1,calm,"[[-902.4064116162402, 6.517241898027468, 6.427..."


In [9]:
df_normal = pd.read_pickle(PROCESSED_DATA_PICKLE_PATH_N)
df_normal.head()

,path,source,actor,gender,gender_label,statement,repetition,intensity,emotion,emotion_label,augmentation,mfcc
0,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,1,0,0,neutral,none,
1,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,2,0,0,neutral,none,
2,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,1,1,0,0,neutral,none,
3,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,1,2,0,0,neutral,none,
4,../data/Audio_Speech_Actors_01-24/Actor_01/03-...,1,1,1,male,0,1,0,1,calm,none,


In [ ]:
def load_data(data_path):
    df = pd.read_pickle(data_path)

    # clean up data
    vals = []
    for i, val in enumerate(df["mfcc"].to_numpy()):
        if len(val) < MFCC_LENGTH:
            vals.append(i)

    df = df.drop(vals)
    df = df.reset_index(drop=True)

    X = np.stack(df["mfcc"].to_numpy())
    y = np.stack(df["emotion"].to_numpy())

    return X, y

In [3]:
def prepare_datasets(test_size):

    # load data
    X, y = load_data(PROCESSED_DATA_PICKLE_PATH)

    new_X = []
    for i in X:
        avg_mfcc = []
        for j in i:
            avg_mfcc.append(j.mean())
        new_X.append(avg_mfcc)
    X = np.array(new_X)

    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = prepare_datasets(0.3)

In [4]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)

print(X_test.shape)

y_pred = clf.predict(X_test)

print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))


(432, 125)
Accuracy: 0.3148148148148148
